In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
# chdir
data_dir = "/home/blue_pc/code/esunbank-ML/raw_data"
os.chdir(data_dir)

# by alert key
train_y = pd.read_csv("train_y_answer.csv", index_col=0)
custinfo = pd.read_csv("public_train_x_custinfo_full_hashed.csv", index_col=0)
alert_date = pd.read_csv("train_x_alert_date.csv", index_col=0)
full_cust_info = pd.concat((custinfo, alert_date, train_y), axis=1, join="inner")

# by cust_id
cdtx = pd.read_csv("public_train_x_cdtx0001_full_hashed.csv", index_col=0)
dp = pd.read_csv("public_train_x_dp_full_hashed.csv", index_col=0)
remit1 = pd.read_csv("public_train_x_remit1_full_hashed.csv", index_col=0)
ccba = pd.read_csv("public_train_x_ccba_full_hashed.csv", index_col=0)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

#　cust前處理
cdtx_drop_date_cust_id = full_cust_info.drop(["cust_id", "date",], axis=1).values
scaler.fit(cdtx_drop_date_cust_id)
full_cust_info.loc[: ,(full_cust_info.columns != "date")&(full_cust_info.columns != "cust_id")] = scaler.transform(cdtx_drop_date_cust_id)

# cdtx前處理

cdtx = cdtx.fillna(0)
cdtx.amt = np.where(cdtx.amt!=0, np.log10(cdtx.amt), 0)
cdtx.country = np.where(cdtx.country==130, 1, 0)
cdtx_drop_date = cdtx.drop("date", axis=1).values
scaler.fit(cdtx_drop_date)
cdtx.loc[: ,cdtx.columns != "date"] = scaler.transform(cdtx_drop_date)

# dp前處理
dp.tx_amt = np.where(dp.tx_amt!=0, np.log10(dp.tx_amt), 0)
sign = np.where(dp.debit_credit == "CR", 1, -1)
dp = dp.drop(["debit_credit", "fiscTxId", "txbranch", "tx_time"], axis=1) # 
dp = dp.rename(columns={'tx_date': 'date'})
dp = dp.fillna(0)
dp_drop_date = dp.drop("date", axis=1).values
scaler.fit(dp_drop_date)
dp.loc[: ,dp.columns != "date"] = scaler.transform(dp_drop_date)
dp.tx_amt = sign*dp.tx_amt # 若為借出則改為一


# remit1前處理
remit1 = remit1.drop("trans_no", axis=1)
remit1 = remit1.rename(columns={'trans_date': 'date'})

remit1.trade_amount_usd = np.where(remit1.trade_amount_usd!=0, np.log10(remit1.trade_amount_usd), 0)
remit1 = remit1.fillna(0)
remit1_drop_date = remit1.drop("date", axis=1).values
scaler.fit(remit1_drop_date)
remit1.loc[: ,remit1.columns != "date"] = scaler.transform(remit1_drop_date)

# 觀察資料 推測與date一樣
ccba = ccba.rename(columns={'byymm': 'date'})
ccba.lupay = np.where(ccba.lupay!=0, np.log10(ccba.lupay), 0)
ccba.cycam = np.where(ccba.cycam!=0, np.log10(ccba.cycam), 0)
ccba = ccba.fillna(0)
ccba_drop_date = ccba.drop("date", axis=1).values
scaler.fit(ccba_drop_date)
ccba.loc[: ,ccba.columns != "date"] = scaler.transform(ccba_drop_date)

In [ ]:
data_dir =  "/home/blue_pc/code/esunbank-ML/raw_data/clean_data"
os.chdir(data_dir)
full_cust_info.to_csv("full_cust_info.csv")
cdtx.to_csv("cdtx.csv")
dp.to_csv("dp.csv")
remit1.to_csv("remit1.csv")
ccba.to_csv("ccba.csv")